In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.ticker import FuncFormatter

# Load the dataset
df = pd.read_csv('airport_traffic_2016_2024_cleaned.csv')

# Basic data check
print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"Years range: {df['YEAR'].min()} to {df['YEAR'].max()}")

# Note: In your dataset, it seems FLT_TOT_1 is used instead of FLT_TOT_J
# and FLT_TOT_IFR_2 for IFR flights.

# ----------------------
# 3. Airport Performance & Ranking
# ----------------------

# 3.1 Which airports grew the most from 2016 to 2024?
# Group by airport and year, and sum flight metrics
airport_yearly = df.groupby(['APT_ICAO', 'APT_NAME', 'STATE_NAME', 'YEAR'])['FLT_TOT_1'].sum().reset_index()

# Create a pivot table for easier comparison
airport_pivot = airport_yearly.pivot_table(
    index=['APT_ICAO', 'APT_NAME', 'STATE_NAME'], 
    columns='YEAR', 
    values='FLT_TOT_1',
    fill_value=0
).reset_index()

# Calculate growth between 2016 and 2024
# Only include airports with data for both years
airport_growth = airport_pivot[airport_pivot[2016] > 0].copy()
airport_growth['growth_absolute'] = airport_growth[2024] - airport_growth[2016]
airport_growth['growth_percentage'] = (airport_growth[2024] / airport_growth[2016] - 1) * 100

# Top 15 airports by absolute growth
top_growth = airport_growth.sort_values('growth_absolute', ascending=False).head(15)
bottom_growth = airport_growth.sort_values('growth_absolute').head(15)

# Visualization: Top airports by growth
plt.figure(figsize=(12, 8))
sns.barplot(x='growth_absolute', y='APT_NAME', data=top_growth, hue='STATE_NAME', palette='viridis', dodge=False)
plt.title('Top 15 Airports by Absolute Growth in Total Flights (2016-2024)', fontsize=14)
plt.xlabel('Growth in Total Flights', fontsize=12)
plt.ylabel('Airport', fontsize=12)
plt.tight_layout()
plt.savefig('top_airports_growth.png')
plt.close()

# Visualization: Bottom airports by growth (most declined)
plt.figure(figsize=(12, 8))
sns.barplot(x='growth_absolute', y='APT_NAME', data=bottom_growth, hue='STATE_NAME', palette='rocket_r', dodge=False)
plt.title('15 Airports with Largest Decline in Total Flights (2016-2024)', fontsize=14)
plt.xlabel('Decline in Total Flights', fontsize=12)
plt.ylabel('Airport', fontsize=12)
plt.tight_layout()
plt.savefig('bottom_airports_growth.png')
plt.close()

# 3.2 Percentage of total traffic handled by top 10 busiest airports over time
# Calculate total traffic by year
total_traffic_by_year = df.groupby('YEAR')['FLT_TOT_1'].sum().reset_index()

# Calculate traffic for top 10 airports by year
# First, identify top 10 airports for each year
top_airports_by_year = {}
for year in df['YEAR'].unique():
    year_data = df[df['YEAR'] == year]
    top_airports = year_data.groupby(['APT_ICAO'])['FLT_TOT_1'].sum().nlargest(10).index.tolist()
    top_airports_by_year[year] = top_airports

# Calculate total traffic for top 10 airports each year
top10_traffic_by_year = []
for year, airports in top_airports_by_year.items():
    traffic = df[(df['YEAR'] == year) & (df['APT_ICAO'].isin(airports))]['FLT_TOT_1'].sum()
    top10_traffic_by_year.append({'YEAR': year, 'TOP10_TRAFFIC': traffic})

top10_df = pd.DataFrame(top10_traffic_by_year)

# Merge with total traffic
traffic_concentration = pd.merge(top10_df, total_traffic_by_year, on='YEAR')
traffic_concentration['TOP10_PERCENTAGE'] = (traffic_concentration['TOP10_TRAFFIC'] / traffic_concentration['FLT_TOT_1']) * 100

# Visualization: Traffic concentration in top 10 airports
plt.figure(figsize=(10, 6))
plt.plot(traffic_concentration['YEAR'], traffic_concentration['TOP10_PERCENTAGE'], 'o-', linewidth=2, markersize=8)
plt.axvline(x=2020, color='r', linestyle='--', alpha=0.7, label='COVID-19 Pandemic')
plt.title('Percentage of Total European Air Traffic Handled by Top 10 Busiest Airports', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Percentage of Total Traffic (%)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(traffic_concentration['YEAR'])
plt.legend()
plt.tight_layout()
plt.savefig('traffic_concentration.png')
plt.close()

# ----------------------
# 4. Commercial vs. Business Flight Trends
# ----------------------

# 4.1 Compare IFR flights to total flights over time
# Group by year
flight_types_yearly = df.groupby('YEAR').agg({
    'FLT_TOT_1': 'sum',
    'FLT_TOT_IFR_2': 'sum'
}).reset_index()

flight_types_yearly['IFR_PERCENTAGE'] = (flight_types_yearly['FLT_TOT_IFR_2'] / flight_types_yearly['FLT_TOT_1']) * 100
flight_types_yearly['NON_IFR_PERCENTAGE'] = 100 - flight_types_yearly['IFR_PERCENTAGE']

# Visualization: IFR vs Total Flights Trend
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(flight_types_yearly['YEAR'], flight_types_yearly['FLT_TOT_1'], 'o-', label='Total Flights', linewidth=2)
plt.plot(flight_types_yearly['YEAR'], flight_types_yearly['FLT_TOT_IFR_2'], 'o-', label='IFR Flights', linewidth=2)
plt.axvline(x=2020, color='r', linestyle='--', alpha=0.5, label='COVID-19')
plt.title('IFR vs Total Flights (2016-2024)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Number of Flights', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(flight_types_yearly['YEAR'])
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(flight_types_yearly['YEAR'], flight_types_yearly['IFR_PERCENTAGE'], 'o-', linewidth=2, color='green')
plt.axvline(x=2020, color='r', linestyle='--', alpha=0.5, label='COVID-19')
plt.title('IFR Flights as Percentage of Total Flights', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Percentage (%)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(flight_types_yearly['YEAR'])
plt.tight_layout()
plt.savefig('ifr_vs_total_flights.png')
plt.close()

# 4.2 Proxy for business vs commercial: Airport size analysis
# We can use airport size as a proxy, assuming smaller airports tend to have more business flights
# Categorize airports by size (small, medium, large) based on total traffic

# Calculate average yearly traffic for each airport
airport_size = df.groupby(['APT_ICAO', 'APT_NAME', 'STATE_NAME'])['FLT_TOT_1'].sum().reset_index()
airport_size['avg_yearly_traffic'] = airport_size['FLT_TOT_1'] / len(df['YEAR'].unique())

# Categorize airports
airport_size['size_category'] = pd.qcut(
    airport_size['avg_yearly_traffic'], 
    q=[0, 0.5, 0.8, 1.0], 
    labels=['Small', 'Medium', 'Large']
)

# Merge airport size back to original data
airport_map = airport_size[['APT_ICAO', 'size_category']].set_index('APT_ICAO').to_dict()['size_category']
df['airport_size'] = df['APT_ICAO'].map(airport_map)

# Analyze IFR percentage by airport size over time
size_ifr_trend = df.groupby(['YEAR', 'airport_size']).agg({
    'FLT_TOT_1': 'sum',
    'FLT_TOT_IFR_2': 'sum'
}).reset_index()

size_ifr_trend['IFR_PERCENTAGE'] = (size_ifr_trend['FLT_TOT_IFR_2'] / size_ifr_trend['FLT_TOT_1']) * 100

# Pivot for visualization
size_ifr_pivot = size_ifr_trend.pivot(
    index='YEAR', 
    columns='airport_size', 
    values='IFR_PERCENTAGE'
)

# Visualization: IFR percentage by airport size (proxy for business vs commercial)
plt.figure(figsize=(10, 6))
size_ifr_pivot.plot(marker='o', linewidth=2)
plt.axvline(x=2020, color='r', linestyle='--', alpha=0.7, label='COVID-19 Pandemic')
plt.title('IFR Flight Percentage by Airport Size (2016-2024)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('IFR Percentage (%)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend(title='Airport Size')
plt.tight_layout()
plt.savefig('ifr_by_airport_size.png')
plt.close()

# 4.3 Regional analysis: Do low-cost carriers or short-haul flights dominate certain regions?
# We can use the IFR to total flight ratio as a proxy, with lower ratios suggesting more non-IFR operations
# (typically associated with low-cost and regional carriers)

# Calculate IFR percentage by country and year
country_ifr = df.groupby(['STATE_NAME', 'YEAR']).agg({
    'FLT_TOT_1': 'sum',
    'FLT_TOT_IFR_2': 'sum'
}).reset_index()

country_ifr['IFR_PERCENTAGE'] = (country_ifr['FLT_TOT_IFR_2'] / country_ifr['FLT_TOT_1']) * 100

# Get top 10 countries by total traffic
top_countries = df.groupby('STATE_NAME')['FLT_TOT_1'].sum().nlargest(10).index.tolist()
country_ifr_filtered = country_ifr[country_ifr['STATE_NAME'].isin(top_countries)]

# Visualization: IFR percentage by country over time
plt.figure(figsize=(14, 8))
for country in top_countries:
    country_data = country_ifr_filtered[country_ifr_filtered['STATE_NAME'] == country]
    plt.plot(country_data['YEAR'], country_data['IFR_PERCENTAGE'], 'o-', linewidth=1.5, label=country)

plt.axvline(x=2020, color='r', linestyle='--', alpha=0.5, label='COVID-19')
plt.title('IFR Flight Percentage by Country (2016-2024)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('IFR Percentage (%)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('ifr_by_country.png')
plt.close()

# Create a summary table for 2023 (most recent complete year)
# Countries with lowest IFR percentage might have more low-cost/short-haul dominance
country_summary_2023 = country_ifr[country_ifr['YEAR'] == 2023].sort_values('IFR_PERCENTAGE')
print("\nCountries with lowest IFR percentage in 2023 (potential low-cost carrier dominance):")
print(country_summary_2023[['STATE_NAME', 'IFR_PERCENTAGE']].head(10))

print("\nCountries with highest IFR percentage in 2023 (potential business/long-haul dominance):")
print(country_summary_2023[['STATE_NAME', 'IFR_PERCENTAGE']].tail(10))

# 4.4 Analyze seasonality patterns for different airport sizes
# This can help identify business vs leisure travel patterns
monthly_by_size = df.groupby(['MONTH_NUM', 'airport_size'])['FLT_TOT_1'].sum().reset_index()
monthly_by_size_pivot = monthly_by_size.pivot(index='MONTH_NUM', columns='airport_size', values='FLT_TOT_1')

# Normalize to show percentage of annual traffic
for col in monthly_by_size_pivot.columns:
    monthly_by_size_pivot[col] = monthly_by_size_pivot[col] / monthly_by_size_pivot[col].sum() * 100

# Visualization: Seasonality patterns by airport size
plt.figure(figsize=(12, 6))
monthly_by_size_pivot.plot(marker='o', linewidth=2)
plt.title('Monthly Traffic Distribution by Airport Size (2016-2024)', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Percentage of Annual Traffic (%)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.legend(title='Airport Size')
plt.tight_layout()
plt.savefig('seasonality_by_airport_size.png')
plt.close()

print("\nAnalysis complete. All visualizations have been saved.")

Dataset shape: (292851, 13)
Columns: ['YEAR', 'MONTH_NUM', 'MONTH_MON', 'FLT_DATE', 'APT_ICAO', 'APT_NAME', 'STATE_NAME', 'FLT_DEP_1', 'FLT_ARR_1', 'FLT_TOT_1', 'FLT_DEP_IFR_2', 'FLT_ARR_IFR_2', 'FLT_TOT_IFR_2']
Years range: 2016 to 2024

Countries with lowest IFR percentage in 2023 (potential low-cost carrier dominance):
         STATE_NAME  IFR_PERCENTAGE
134          Latvia       96.460322
25         Bulgaria       98.290399
225           Spain       98.404532
207        Slovakia       98.503766
125           Italy       99.117036
44   Czech Republic       99.222581
80           France       99.254932
71          Finland       99.351464
146      Luxembourg       99.362774
165     Netherlands       99.399803

Countries with highest IFR percentage in 2023 (potential business/long-haul dominance):
         STATE_NAME  IFR_PERCENTAGE
256  United Kingdom       99.997684
191        Portugal       99.997735
200         Romania      100.030545
247          Turkey      100.190618
98         

<Figure size 1000x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import matplotlib.dates as mdates
from datetime import datetime

# Load the dataset
df = pd.read_csv('airport_traffic_2016_2024_cleaned.csv')

# Define pandemic periods for reference
pre_pandemic = [2016, 2017, 2018, 2019]
pandemic = [2020, 2021]
post_pandemic = [2022, 2023, 2024]

# Basic data check
print(f"Dataset shape: {df.shape}")
print(f"Years range: {df['YEAR'].min()} to {df['YEAR'].max()}")

# ----------------------
# 5. Post-Pandemic Recovery Analysis
# ----------------------

# 5.1 Compare flight volumes before, during, and after pandemic
yearly_traffic = df.groupby('YEAR')['FLT_TOT_1'].sum().reset_index()

# Calculate year-over-year percentage changes
yearly_traffic['YoY_Change'] = yearly_traffic['FLT_TOT_1'].pct_change() * 100

# Calculate percentage vs. 2019 (last pre-pandemic year)
pre_pandemic_max = yearly_traffic[yearly_traffic['YEAR'] == 2019]['FLT_TOT_1'].values[0]
yearly_traffic['Pct_vs_2019'] = (yearly_traffic['FLT_TOT_1'] / pre_pandemic_max * 100) - 100

# Create period labels for visualization
yearly_traffic['Period'] = yearly_traffic['YEAR'].apply(
    lambda x: 'Pre-Pandemic' if x in pre_pandemic else 
              'Pandemic' if x in pandemic else 'Post-Pandemic'
)

# Visualization: Flight volumes comparison
plt.figure(figsize=(12, 6))
ax = sns.barplot(x='YEAR', y='FLT_TOT_1', data=yearly_traffic, 
                hue='Period', palette={'Pre-Pandemic':'#1f77b4', 'Pandemic':'#d62728', 'Post-Pandemic':'#2ca02c'})

# Add percentage vs 2019 above each bar
for i, row in enumerate(yearly_traffic.itertuples()):
    if row.YEAR != 2019:  # Skip 2019 as it's the reference year (0%)
        plt.text(i, row.FLT_TOT_1 + 2000000, f"{row.Pct_vs_2019:.1f}%", 
                ha='center', va='bottom', fontsize=9,
                color='green' if row.Pct_vs_2019 > 0 else 'red')
    else:
        plt.text(i, row.FLT_TOT_1 + 2000000, "Ref", ha='center', va='bottom', fontsize=9)

plt.axhline(y=pre_pandemic_max, color='black', linestyle='--', alpha=0.5, label='2019 Level')
plt.title('Total Flight Volumes Before, During, and After Pandemic', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Total Number of Flights', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('pandemic_flight_volumes.png')
plt.close()

# 5.2 Which airports recovered fastest post-pandemic?
# First, get airport traffic for 2019 (pre-pandemic reference) and 2022 (first full recovery year)
airport_2019 = df[df['YEAR'] == 2019].groupby(['APT_ICAO', 'APT_NAME', 'STATE_NAME'])['FLT_TOT_1'].sum().reset_index()
airport_2022 = df[df['YEAR'] == 2022].groupby(['APT_ICAO', 'APT_NAME', 'STATE_NAME'])['FLT_TOT_1'].sum().reset_index()
airport_2023 = df[df['YEAR'] == 2023].groupby(['APT_ICAO', 'APT_NAME', 'STATE_NAME'])['FLT_TOT_1'].sum().reset_index()
airport_2024 = df[df['YEAR'] == 2024].groupby(['APT_ICAO', 'APT_NAME', 'STATE_NAME'])['FLT_TOT_1'].sum().reset_index()

# Merge to calculate recovery percentages
recovery_2022 = pd.merge(airport_2019, airport_2022, on=['APT_ICAO', 'APT_NAME', 'STATE_NAME'], suffixes=('_2019', '_2022'))
recovery_2022['recovery_pct'] = (recovery_2022['FLT_TOT_1_2022'] / recovery_2022['FLT_TOT_1_2019']) * 100

recovery_2023 = pd.merge(airport_2019, airport_2023, on=['APT_ICAO', 'APT_NAME', 'STATE_NAME'], suffixes=('_2019', '_2023'))
recovery_2023['recovery_pct'] = (recovery_2023['FLT_TOT_1_2023'] / recovery_2023['FLT_TOT_1_2019']) * 100

recovery_2024 = pd.merge(airport_2019, airport_2024, on=['APT_ICAO', 'APT_NAME', 'STATE_NAME'], suffixes=('_2019', '_2024'))
recovery_2024['recovery_pct'] = (recovery_2024['FLT_TOT_1_2024'] / recovery_2024['FLT_TOT_1_2019']) * 100

# Filter for airports with significant traffic (avoid small airports with extreme percentages)
min_traffic = 10000  # Minimum annual flights to be considered
recovery_2022 = recovery_2022[recovery_2022['FLT_TOT_1_2019'] > min_traffic]
recovery_2023 = recovery_2023[recovery_2023['FLT_TOT_1_2019'] > min_traffic]
recovery_2024 = recovery_2024[recovery_2024['FLT_TOT_1_2019'] > min_traffic]

# Top 15 fastest recovered airports by 2022
top_recovery_2022 = recovery_2022.sort_values('recovery_pct', ascending=False).head(15)
top_recovery_2022['Year'] = 2022

# Top 15 fastest recovered airports by 2023
top_recovery_2023 = recovery_2023.sort_values('recovery_pct', ascending=False).head(15)
top_recovery_2023['Year'] = 2023

# Top 15 fastest recovered airports by 2024
top_recovery_2024 = recovery_2024.sort_values('recovery_pct', ascending=False).head(15)
top_recovery_2024['Year'] = 2024

# Visualization: Top recovered airports in 2022
plt.figure(figsize=(12, 8))
sns.barplot(x='recovery_pct', y='APT_NAME', hue='STATE_NAME', 
           data=top_recovery_2022, palette='viridis', dodge=False)
plt.axvline(x=100, color='red', linestyle='--', label='2019 Level')
plt.title('Top 15 Airports with Fastest Recovery by 2022 (vs. 2019)', fontsize=14)
plt.xlabel('Recovery Percentage (%)', fontsize=12)
plt.ylabel('Airport', fontsize=12)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('top_recovery_airports_2022.png')
plt.close()

# Visualization: Top recovered airports in 2024
plt.figure(figsize=(12, 8))
sns.barplot(x='recovery_pct', y='APT_NAME', hue='STATE_NAME', 
           data=top_recovery_2024, palette='viridis', dodge=False)
plt.axvline(x=100, color='red', linestyle='--', label='2019 Level')
plt.title('Top 15 Airports with Fastest Recovery by 2024 (vs. 2019)', fontsize=14)
plt.xlabel('Recovery Percentage (%)', fontsize=12)
plt.ylabel('Airport', fontsize=12)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('top_recovery_airports_2024.png')
plt.close()

# 5.3 Were there lasting changes in seasonal travel peaks?
# Compare monthly patterns before and after pandemic
df['Period'] = df['YEAR'].apply(
    lambda x: 'Pre-Pandemic' if x in pre_pandemic else 
              'Pandemic' if x in pandemic else 'Post-Pandemic'
)

# Exclude pandemic period for clearer before/after comparison
seasonal_df = df[df['Period'] != 'Pandemic']

# Group by period and month
seasonal_pattern = seasonal_df.groupby(['Period', 'MONTH_NUM'])['FLT_TOT_1'].sum().reset_index()

# Calculate percentage of annual traffic by period
period_totals = seasonal_pattern.groupby('Period')['FLT_TOT_1'].sum()
seasonal_pattern['monthly_pct'] = seasonal_pattern.apply(
    lambda x: (x['FLT_TOT_1'] / period_totals[x['Period']]) * 100, axis=1
)

# Pivot for easier plotting
seasonal_pivot = seasonal_pattern.pivot(index='MONTH_NUM', columns='Period', values='monthly_pct')

# Visualization: Seasonal patterns before and after pandemic
plt.figure(figsize=(10, 6))
seasonal_pivot.plot(marker='o', linewidth=2)
plt.title('Monthly Traffic Distribution Before and After Pandemic', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Percentage of Annual Traffic (%)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.legend(title='Period')
plt.tight_layout()
plt.savefig('seasonal_patterns_change.png')
plt.close()

# ----------------------
# 6. COVID-19 Impact Analysis
# ----------------------

# 6.1 Pre-Pandemic Growth Analysis (2016-2019)
pre_pandemic_trend = yearly_traffic[yearly_traffic['YEAR'].isin(pre_pandemic)]

# Calculate compound annual growth rate (CAGR) for pre-pandemic period
first_year = pre_pandemic_trend['FLT_TOT_1'].iloc[0]
last_year = pre_pandemic_trend['FLT_TOT_1'].iloc[-1]
years = len(pre_pandemic_trend) - 1
pre_pandemic_cagr = (last_year / first_year) ** (1/years) - 1

# 6.2 Post-Pandemic Growth Analysis (2022-2024)
post_pandemic_trend = yearly_traffic[yearly_traffic['YEAR'].isin(post_pandemic)]

# Calculate compound annual growth rate (CAGR) for post-pandemic period
first_year_post = post_pandemic_trend['FLT_TOT_1'].iloc[0]
last_year_post = post_pandemic_trend['FLT_TOT_1'].iloc[-1]
years_post = len(post_pandemic_trend) - 1
post_pandemic_cagr = (last_year_post / first_year_post) ** (1/years_post) - 1

# Print CAGR comparison
print(f"Pre-pandemic CAGR (2016-2019): {pre_pandemic_cagr*100:.2f}%")
print(f"Post-pandemic CAGR (2022-2024): {post_pandemic_cagr*100:.2f}%")

# Calculate if 2024 levels exceeded 2019
recovery_complete = last_year_post >= last_year
print(f"Has air traffic fully recovered to pre-pandemic levels? {'Yes' if recovery_complete else 'No'}")
print(f"2024 traffic is {last_year_post/last_year*100:.2f}% of 2019 traffic")

# Visualization: Growth trends with pandemic impact
plt.figure(figsize=(12, 6))

# Plot actual data
plt.plot(yearly_traffic['YEAR'], yearly_traffic['FLT_TOT_1'], 'o-', color='blue', linewidth=2, label='Actual Traffic')

# Extend pre-pandemic trend line to show what might have happened without pandemic
x_pre = np.array(pre_pandemic)
y_pre = pre_pandemic_trend['FLT_TOT_1'].values


# Calculate trend line with linear regression
z_pre = np.polyfit(x_pre, y_pre, 1)
p_pre = np.poly1d(z_pre)

# Create extended x-axis for prediction
x_extended = np.array(range(2016, 2025))
y_extended = p_pre(x_extended)

# Plot pre-pandemic trend line extended
plt.plot(x_extended, y_extended, '--', color='green', linewidth=2, alpha=0.7, 
         label=f'Pre-Pandemic Trend (CAGR: {pre_pandemic_cagr*100:.2f}%)')

# Highlight 2019 reference level
plt.axhline(y=last_year, color='red', linestyle='--', alpha=0.5, label='2019 Level')

# Add pandemic period shading
plt.axvspan(2020, 2021, color='red', alpha=0.2, label='Pandemic Period')

# Add post-pandemic growth rate annotation
plt.annotate(f'Post-Pandemic CAGR: {post_pandemic_cagr*100:.2f}%', 
             xy=(2023, post_pandemic_trend['FLT_TOT_1'].iloc[-2]),
             xytext=(2021, post_pandemic_trend['FLT_TOT_1'].iloc[-2] * 0.8),
             arrowprops=dict(facecolor='black', shrink=0.05, width=1.5, headwidth=8),
             fontsize=10)

# Add gap between actual and projected
if y_extended[-1] > last_year_post:
    gap_pct = (y_extended[-1] - last_year_post) / y_extended[-1] * 100
    plt.annotate(f'Gap: {gap_pct:.1f}%', 
                xy=(2024, (y_extended[-1] + last_year_post)/2),
                xytext=(2022.5, (y_extended[-1] + last_year_post)/2),
                arrowprops=dict(facecolor='black', shrink=0.05, width=1.5, headwidth=8),
                fontsize=10)

plt.title('European Air Traffic Evolution: Impact of COVID-19 Pandemic (2016-2024)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Total Number of Flights', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(yearly_traffic['YEAR'])
plt.legend()
plt.tight_layout()
plt.savefig('pandemic_impact_analysis.png')
plt.close()

# 6.3 Country-Level Recovery Analysis
# Get country-level data for 2019 and 2024
country_2019 = df[df['YEAR'] == 2019].groupby('STATE_NAME')['FLT_TOT_1'].sum().reset_index()
country_2024 = df[df['YEAR'] == 2024].groupby('STATE_NAME')['FLT_TOT_1'].sum().reset_index()

# Calculate recovery percentages
country_recovery = pd.merge(country_2019, country_2024, on='STATE_NAME', suffixes=('_2019', '_2024'))
country_recovery['recovery_pct'] = (country_recovery['FLT_TOT_1_2024'] / country_recovery['FLT_TOT_1_2019']) * 100

# Sort by recovery percentage
country_recovery_sorted = country_recovery.sort_values('recovery_pct', ascending=False)

# Visualization: Country recovery levels
plt.figure(figsize=(12, 8))
ax = sns.barplot(x='recovery_pct', y='STATE_NAME', data=country_recovery_sorted, palette='viridis')
plt.axvline(x=100, color='red', linestyle='--', label='2019 Level')

# Add traffic volume as text on bars
for i, row in enumerate(country_recovery_sorted.itertuples()):
    plt.text(105, i, f"{row.FLT_TOT_1_2024:,.0f} flights", va='center', fontsize=8)

plt.title('Country-Level Recovery: 2024 Traffic as Percentage of 2019 Levels', fontsize=14)
plt.xlabel('Recovery Percentage (%)', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('country_recovery_levels.png')
plt.close()

print("\nAnalysis complete. All visualizations have been saved.")

Dataset shape: (292851, 13)
Years range: 2016 to 2024
Pre-pandemic CAGR (2016-2019): 10.47%
Post-pandemic CAGR (2022-2024): 8.31%
Has air traffic fully recovered to pre-pandemic levels? No
2024 traffic is 94.34% of 2019 traffic


C:\Users\Prajwal Bang\AppData\Local\Temp\ipykernel_10340\2122968595.py:260: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.barplot(x='recovery_pct', y='STATE_NAME', data=country_recovery_sorted, palette='viridis')



Analysis complete. All visualizations have been saved.


<Figure size 1000x600 with 0 Axes>